In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# Import data
df = pd.read_csv('data/pokemon.csv')

# For columns with many classes, we will get dummy variable for each class
oh_cols = ['type1', 'type2', 'generation'] # Columns that need to be encoded with OH
dum_df = pd.get_dummies(df, columns=oh_cols, prefix=['type1_', 'type2_', 'generation_'])
df = df.join(dum_df, rsuffix='_dum')
# Drop duplicate columns
df.drop([col for col in df.columns if '_dum' in col], axis=1, inplace=True)

# TODO make columns for greater or less than mean for hp, attack, anc defense

# Drop non-binary columns
df.drop(['classification', 'type1','type2', 'attack', 'height_m', 'hp','percentage_male','speed','weight_kg'], axis=1, inplace=True)

# Set default values for missing values
df.fillna(df.mean(), inplace=True) # TODO adjest mean here

# Assign id to each pokemon that we can map back to its name
df['id'] = np.arange(1, len(df)+1)

print(df)
df.to_csv('data/pokemon_preprocessed.csv', index=False)

           name  generation  is_legendary  type1__bug  type1__dark  \
0     Bulbasaur           1             0           0            0   
1       Ivysaur           1             0           0            0   
2      Venusaur           1             0           0            0   
3    Charmander           1             0           0            0   
4    Charmeleon           1             0           0            0   
..          ...         ...           ...         ...          ...   
796  Celesteela           7             1           0            0   
797     Kartana           7             1           0            0   
798    Guzzlord           7             1           0            1   
799    Necrozma           7             1           0            0   
800    Magearna           7             1           0            0   

     type1__dragon  type1__electric  type1__fairy  type1__fighting  \
0                0                0             0                0   
1                0 

C:\Users\wilmc\AppData\Local\Temp/ipykernel_16972/1972136863.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True) # TODO adjest mean here
